# CSC324 Lecture5

In [1]:
import Data.Maybe

## Functors

### Motivation
Recall the mapping function `map` described in the previous lecture

In [2]:
fmap_List :: (a -> b) -> [] a -> [] b
-- Take a list of a and convert it to a list of b
fmap_List f [] = []
fmap_List f (x:xs) = f x : fmap_List f xs

Look at the signature. Notice `[] a -> [] b`.  
It's just `[a] -> [b]`.  
We already saw myMap and map functions

LetConsider the data structure `Maybe` defined from the standard library. We already saw it on the assignment.
```
data Maybe a = Nothing | Just a deriving Show
```
You can think of it as either there is an answer or there no answer.  
Suppose we want to define `fmap` for this data structure.

In [3]:
fmap_Maybe :: (a -> b) -> Maybe a -> Maybe b
fmap_Maybe f Nothing = Nothing -- There is no answer in this circumstance
fmap_Maybe f (Just a) = Just (f a) -- There is an answer and you map it to an answer

So in this case, we don't use recursion and we define it for all the cases of the definition.

Similarly, we can do the same for the data structure `Either`.  
The motivation behind `Either` is that maybe when we have no answer, we don't want to return *nothing*. Thus, we may want to use `Either` for the case where we want to return something else when there is no answer

In [4]:
data Either e a = Left e | Right a

Again, let's define an `fmap` function similar to how we defined for the above

In [5]:
fmap_Either :: (a -> b) -> (Either e) a -> (Either e) b
fmap_Either f (Left e) = Left e -- This is like nothing.
-- We just return Left e, which is essentially nothing
fmap_Either f (Right a) = Right (f a)

We can do the same for other data structures. But the issue is that we have to assign different names for `fmap`.  
Like above, we defined `fmap_List`, `fmap_Maybe` and `fmap_Either` and they basically just do `fmap`.  
If only we could make these data structures inherit some class so that they can use the same function name. Like using `fmap` for `Either` and `Maybe` rather than introducing new function names.  
This is where `Functors` play a role.

### What are Functors ?
Functor in Haskell is a kind of functional representation of different Types which can be mapped over. It is a high level concept of implementing polymorphism.
The common theme is `(a -> b) -> (F a -> F b)` where `F` is a parametrized type.  
The idea is that a data structure `F` that is an instance of `Functor` has a function fmap which takes in 2 arguments a function `f` that maps `a` to `b` and returns a data structure that supports mapping. That is, it will return a function that maps `F a` to `F b`.  

![](https://bartoszmilewski.files.wordpress.com/2015/01/functor.jpg)

### How to use them

In [6]:
class Functor f where -- Notice we don't put (f a) so that 
                      -- it is generic (kinda)
       fmap :: (a -> b) -> (f a -> f b)
        -- You can define other functions here.

Then for any data structure, we have to make sure that it is an **instance** of `Functor` so that we can use `fmap` 
```
   instance Functor [] where
       fmap = map

   instance Functor Maybe where -- Maybe nature rather than
                                -- what kind of elements you have
       fmap = ... like fmap_Maybe above

   instance Functor (Either e) where
       fmap = ... like fmap_Either above
```

Something important to note is that this is not like generics. For example in Java, if we have generics then we can say `Class<A>`. Here, the idea is that we're **generalizing** for all classes.  
However, here, `Functors` tries to generalize all `A` so that `Class` works.  
**Prof said the above but I'm not sure what it means lol**  
All you need to know is that `Functors` are not like `Generics`

####  Example: Functors on Binary Tree

In [7]:
data BinTree a = BTNil | BTNode a (BinTree a) (BinTree a) deriving Show

In [8]:
instance Functor BinTree where
    -- fmap :: (a -> b) -> BinTree a -> BinTree b
    fmap f BTNil = BTNil
    fmap f (BTNode a lt rt) = BTNode (f a) (fmap f lt) (fmap f rt)

Thus, now `fmap` will work for `BinTree` of any type.  

In [9]:
binTree = BTNode 5 (BTNode 2 (BTNode 1 BTNil BTNil) (BTNode 3 BTNil BTNil)) (BTNode 7 (BTNode 6 BTNil BTNil) (BTNode 8 BTNil BTNil))
binTree

BTNode 5 (BTNode 2 (BTNode 1 BTNil BTNil) (BTNode 3 BTNil BTNil)) (BTNode 7 (BTNode 6 BTNil BTNil) (BTNode 8 BTNil BTNil))

In [10]:
fmap (+ 10) binTree

BTNode 15 (BTNode 12 (BTNode 11 BTNil BTNil) (BTNode 13 BTNil BTNil)) (BTNode 17 (BTNode 16 BTNil BTNil) (BTNode 18 BTNil BTNil))

Notice that this is not like `Collection<Integer>` in Java.  
If it were then `<A, B> fmap` is a function that takes in `A, B` and `Collection<A>` and returns `Collection<B>`.  
The main issue is that `Collection<B>` does imply that `Collection<A>` is of similar instance/class  
Whereas in Haskell `(f a -> f b)`, it is guaranteed that `f a` is of the same type `f b`

Another advantage of `Functor` is that if we have a commond function among different data structures, then we can just define one function for all classes that extend `Functor`  
```
class Functor f where 
       fmap :: (a -> b) -> (f a -> f b)
        commonFunction2 = ... 
        commonFunction3 = ...
```

`Functors` might not be very practical at the moment apart from just having a common function name for different data types. But in category theory it is quite useful.  
The main motivation behind this is that we can define one function for all classes that extend `Functor` rather than having different names for the same functions.

It will also be useful when we talk about `Monads`

### Applicative Functors
The standard `map` can apply a unary function *(Function that takes in one argument)* elementwise to one list (or a maybe, or...).  

But, What if you want to apply a binary function to all pairs of elements of two lists (or two maybes, or...)?

In [11]:
map2_List :: (a -> b -> c) -> [a] -> [b] -> [c]
map2_List f [] _ = []
map2_List f (a:as) bs = map (f a) bs ++ map2_List f as bs

In [12]:
map2_List (-) [10, 20, 30] [1, 2, 3]
-- = [9, 8, 7] ++ [19, 18, 17] ++ [29, 28, 27]

[9,8,7,19,18,17,29,28,27]

Here's how the function works

In [13]:
-- How the function works for the above example
"First List"
map (10 -) [1, 2, 3]
"Second list"
map (20 -) [1, 2, 3]
"Third List"
map (30 -) [1, 2, 3]
"Append them all"
map (10 -) [1, 2, 3] ++ map (20 -) [1, 2, 3] ++ map (30 -) [1, 2, 3]

"First List"

[9,8,7]

"Second list"

[19,18,17]

"Third List"

[29,28,27]

"Append them all"

[9,8,7,19,18,17,29,28,27]

Similarly, we can do for `Maybe`

In [14]:
map2_Maybe :: (a -> b -> c) -> Maybe a -> Maybe b -> Maybe c
map2_Maybe f Nothing _ = Nothing
map2_Maybe f (Just a) Nothing = Nothing
map2_Maybe f (Just a) (Just b) = Just ((f a) b)

In [15]:
map2_Maybe (-) Nothing Nothing
map2_Maybe (-) Nothing (Just 5)
map2_Maybe (-) (Just 4) Nothing
map2_Maybe (-) (Just 4) (Just 3)

Nothing

Nothing

Nothing

Just 1

### Having multiple functions for mapping

Now what if we want the above, but this time 3 functions are applied for the lists, or even 4 functions. 
Here's how we would approach it. One of the arguments will be a list of functions

In [16]:
ap_List :: [a -> b] -> [a] -> [b]
ap_List [] as = []
ap_List (f:fs) as = map f as ++ ap_List fs as 
-- Note that (f:fs) is a list of functions

One way to understand this is to work out the intermediate types:  
```
   ap_List    (fmap       f          as)         bs
                      a->(b->c)      [a]
              |<--    [b -> c]      -->|         [b]
   |<---                   [c]                 --->|

```

In [17]:
ap_List [(10 -), (20 -), (30 -)] [1, 2, 3]

[9,8,7,19,18,17,29,28,27]

TODO: Complete the other two functions and introduce `pure`